# Transfer Learning 

Pretrain a CNN on a very large dataset and then use the CNN to initialize or freeze parts of the loaded models

Transfer Learning: https://builtin.com/data-science/transfer-learning 

tochvision.models: https://pytorch.org/vision/stable/models.html

ImageNet data: https://www.image-net.org/ 

Resnet-18: https://arxiv.org/pdf/1512.03385.pdf

In [1]:
def print_accuracy(model, test_loader) : 
  # Leng Lohanakakul 10/25/2022 
  # This function prints the accuracy of the output to the test/target image
  # Function inputs: 
  # model: a CNN model to be evaluated 
  # test_loader: minibatches from the test dataset

  device = torch.device('cuda') 
  model.to(device) # move from CPU to GPU 
  correct, total = 0, 0 
  with torch.no_grad() : 
    for data in test_loader : 
      images, labels = data 
      images = images.to(device)
      labels = labels.to(device) 
      outputs = model(images) # output from the model
      _, predicted = torch.max(outputs.data, 1) # select the highest output data to be evaluated 
      total += labels.size(0) 
      correct += (predicted == labels).sum().item() 
  print('Accuracy of neural network on the test images: %d %%' % (100 * correct / total))

In [8]:
import torch 
import torchvision 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import torchvision.models as models
import torchvision.transforms as transforms 
import time 

# load the pretrained ResNet-18 that is already trained on ImageNet dataset using torchvision.models
resnet18 = models.resnet18(pretrained=True) 

# import and download CIFAR 10 dataset  
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)), torchvision.transforms.Resize((224,224))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# configure loader for each train/test set
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True)

# accuracy of neural network is 0% because the trainset and testset is not from the same dataset
# ResNet-18 is trained on ImageNet but we're testing on CIFAR10 dataset resulting in poor accuracy
print('\n') 
print_accuracy(resnet18, test_loader)

# define stochastic gradient descent optimizer and cross entropy loss function
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9) 
# nn.CrossEntropyLoss function is a combination of softmax function and negative log likelihood
lossFunc = nn.CrossEntropyLoss() 
device = torch.device('cuda') 
epoch_range = 3

for epoch in range(epoch_range): 
  for i, (images, labels) in enumerate(train_loader) : 
    # sends the image and labels to GPU
    images = images.to(device) 
    labels = labels.to(device) 
    # forward propagation 
    outputs = resnet18(images) 
    loss = lossFunc(outputs, labels) 
    # Backward propagation and optimize the model
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 
    # print every 625 mini-batches
    if (i + 1) % 625 == 0: 
      print("Epoch [{}/{}], Step [{}/{}] Loss: {:.3f}".format(epoch+1, epoch_range, i+1, len(train_loader), loss.item()))
    
print_accuracy(resnet18, test_loader)

Files already downloaded and verified
Files already downloaded and verified


Accuracy of neural network on the test images: 0 %
Epoch [1/3], Step [625/3125] Loss: 0.319
Epoch [1/3], Step [1250/3125] Loss: 0.376
Epoch [1/3], Step [1875/3125] Loss: 0.110
Epoch [1/3], Step [2500/3125] Loss: 0.442
Epoch [1/3], Step [3125/3125] Loss: 0.382
Epoch [2/3], Step [625/3125] Loss: 0.164
Epoch [2/3], Step [1250/3125] Loss: 0.060
Epoch [2/3], Step [1875/3125] Loss: 0.401
Epoch [2/3], Step [2500/3125] Loss: 0.129
Epoch [2/3], Step [3125/3125] Loss: 0.151
Epoch [3/3], Step [625/3125] Loss: 0.237
Epoch [3/3], Step [1250/3125] Loss: 0.098
Epoch [3/3], Step [1875/3125] Loss: 0.007
Epoch [3/3], Step [2500/3125] Loss: 0.088
Epoch [3/3], Step [3125/3125] Loss: 0.021
Accuracy of neural network on the test images: 92 %
